# Lauren Bassett 
# jme9rt 
# DS5001 Final Project  

*The goal of your final project is to apply and integrate what you have learned in this course to
create ​a digital critical edition of a corpus​ that will support exploration of the cultural contents of
that corpus. Cultural content is broadly conceived​—it may be about language use, social events,
cultural categories, sentiments, identity, taste, etc.*

## Part 0: Import Statements/Reading in Files

In [1]:
import pandas as pd 
import numpy as np
%matplotlib inline
import re
from glob import glob
import re
import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
xmas_carol_text = 'christmas-carol.txt'
OHCO = ['title', 'chap_num', 'para_num', 'sent_num', 'token_num']

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\laure\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\laure\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\laure\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Part 1: Convert

In [2]:
xmas = open(xmas_carol_text, 'r', encoding='utf-8-sig').readlines()
df = pd.DataFrame(xmas, columns=['line_str'])
df.index.name = 'line_num'
df.line_str = df.line_str.str.strip()
title = df.loc[0].line_str.replace('The Project Gutenberg EBook of ', '')
df['title'] = title

There are illustrations in the text, and I want to make sure to remove the captions for all the illustrations. However, there are a handful that take up multiple lines. To remove these from the corpus, I found it easier to remove these first, then continue with the rest of the data cleansing. 

In [3]:
start = df.line_str.str.match(r'\[.*[^\]]$')
df.loc[start]

,line_str,title
line_num,,
40,"[Illustration: _""How now?"" said Scrooge, caust...","A Christmas Carol, by Charles Dickens"
570,[Illustration: _Bob Cratchit went down a slide...,"A Christmas Carol, by Charles Dickens"
669,[Illustration: _Nobody under the bed; nobody i...,"A Christmas Carol, by Charles Dickens"
971,[Illustration: _The air was filled with phanto...,"A Christmas Carol, by Charles Dickens"
1364,[Illustration: HE PRODUCED A DECANTER OF CURIO...,"A Christmas Carol, by Charles Dickens"
1463,[Illustration: _Then old Fezziwig stood out to...,"A Christmas Carol, by Charles Dickens"
2458,[Illustration: _The way he went after that plu...,"A Christmas Carol, by Charles Dickens"
2735,[Illustration:,"A Christmas Carol, by Charles Dickens"
3527,"[Illustration: _""It's I, your uncle Scrooge. I...","A Christmas Carol, by Charles Dickens"


In [4]:
big_quotes = []
for index, row in df.loc[start].iterrows():
    big_quotes.append(index)
    end = 1
    while end != -1: 
        #Continue to iterate through each line of the dataframe until the illustration is over, which is marked with the ']' character. 
        # Keep track of each line included, all of these need to be removed. 
        big_quotes.append(index+end)
        if re.search('\]', df.iloc[index+end]['line_str']):
            end = -1
        else:
            end = end+1
df.drop(big_quotes, axis=0, inplace=True)
df

,line_str,title
line_num,,
0,The Project Gutenberg EBook of A Christmas Car...,"A Christmas Carol, by Charles Dickens"
1,,"A Christmas Carol, by Charles Dickens"
2,This eBook is for the use of anyone anywhere a...,"A Christmas Carol, by Charles Dickens"
3,almost no restrictions whatsoever. You may co...,"A Christmas Carol, by Charles Dickens"
4,re-use it under the terms of the Project Guten...,"A Christmas Carol, by Charles Dickens"
...,...,...
3971,,"A Christmas Carol, by Charles Dickens"
3972,This Web site includes information about Proje...,"A Christmas Carol, by Charles Dickens"
3973,including how to make donations to the Project...,"A Christmas Carol, by Charles Dickens"


In [5]:
a = df.line_str.str.match(r"\*\*\*\s*START OF (THE|THIS) PROJECT")
b = df.line_str.str.match(r"\*\*\*\s*END OF (THE|THIS) PROJECT")
an = df.loc[a].index[0]
bn = df.loc[b].index[0]
df = df.loc[an + 1 : bn - 2]

What is a Stave? 


"This is an unusual structure that mimics the way a musical piece is put together. The Staves follow the action of the story with the first stave setting the scene, the middle stave showing the turning point for Scrooge and the final stave concluding the story by presenting him as a changed man." -https://www.bbc.co.uk/bitesize/guides/zqxp4j6/revision/2


In [6]:
chap_lines = df.line_str.str.match(r"^\s*(STAVE )(ONE|TWO|THREE|FOUR|FIVE)(?!—)", case=False)
df.loc[chap_lines]

#Remove everything before STAVE ONE
chap_nums = [i+1 for i in range(df.loc[chap_lines].shape[0])]
df.loc[chap_lines, 'chap_num'] = chap_nums
df.chap_num = df.chap_num.ffill()
#Remove all lines containing [Illustration]


C:\Users\laure\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\laure\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, p

In [7]:
df = df.loc[~df.chap_num.isna()] # Remove chapter heading lines
df = df.loc[~chap_lines] # Remove everything before Chapter 1
df.chap_num = df.chap_num.astype('int') # Convert chap_num from float to int

illustration_phrase =  df.line_str.str.match(r'\[Illustration:?.*| ?.*\]')
 #Remove all lines containing [Illustration]
df = df.loc[~illustration_phrase]

In [8]:
dfc = df.groupby(OHCO[:2]).line_str.apply(lambda x: '\n'.join(x)).to_frame() # Make big string

In [9]:
dfp = dfc['line_str'].str.split(r'\n\n+', expand=True).stack()\
    .to_frame().rename(columns={0:'para_str'})
dfp.index.names = OHCO[:3]

dfp['para_str'] = dfp['para_str'].str.replace(r'\n', ' ').str.strip()
dfp = dfp[~dfp['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs

C:\Users\laure\AppData\Local\Temp/ipykernel_24644/4088490130.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  dfp['para_str'] = dfp['para_str'].str.replace(r'\n', ' ').str.strip()


In [17]:
TOKEN = tokenize(dfp, ws=False)

In [13]:
dfs = dfp['para_str'].str.split(r'[.?!;:"]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sent_str'})

dfs.index.names = OHCO[:4]

dfs = dfs[~dfs['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs

In [44]:
dft = dfs['sent_str'].str.split(r"[\s',-]+", expand=True).stack()\
    .to_frame().rename(columns={0:'token_str'})
    
dft.index.names = OHCO[:5]
dft

## Part 2: Annotate

In [19]:
def tokenize(doc_df, OHCO=OHCO, remove_pos_tuple=False, ws=False):
    
    # Paragraphs to Sentences
    df = doc_df.para_str\
        .apply(lambda x: pd.Series(nltk.sent_tokenize(x)))\
        .stack()\
        .to_frame()\
        .rename(columns={0:'sent_str'})
    
    # Sentences to Tokens
    # Local function to pick tokenizer
    def word_tokenize(x):
        if ws:
            s = pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x)))
        else:
            s = pd.Series(nltk.pos_tag(nltk.word_tokenize(x))) # Discards stuff in between
        return s
            
    df = df.sent_str\
        .apply(word_tokenize)\
        .stack()\
        .to_frame()\
        .rename(columns={0:'pos_tuple'})
    
    # Grab info from tuple
    df['pos'] = df.pos_tuple.apply(lambda x: x[1])
    df['token_str'] = df.pos_tuple.apply(lambda x: x[0])
    if remove_pos_tuple:
        df = df.drop('pos_tuple', 1)
    
    # Add index
    df.index.names = OHCO
    
    return df

In [21]:
TOKEN = tokenize(dfp, ws=False)

In [23]:
TOKEN['term_str'] = TOKEN['token_str'].str.lower().str.replace('[\W_]', '')


C:\Users\laure\AppData\Local\Temp/ipykernel_24644/2045287459.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  TOKEN['term_str'] = TOKEN['token_str'].str.lower().str.replace('[\W_]', '')


In [24]:
VOCAB = TOKEN.term_str.value_counts().to_frame()\
    .rename(columns={'index':'term_str', 'term_str':'n'})\
    .sort_index().reset_index().rename(columns={'index':'term_str'})
VOCAB.index.name = 'term_id'
VOCAB['num'] = VOCAB.term_str.str.match("\d+").astype('int')
sw = pd.DataFrame(nltk.corpus.stopwords.words('english'), columns=['term_str'])
sw = sw.reset_index().set_index('term_str')
sw.columns = ['dummy']
sw.dummy = 1

In [25]:
VOCAB['stop'] = VOCAB.term_str.map(sw.dummy)
VOCAB['stop'] = VOCAB['stop'].fillna(0).astype('int')
VOCAB[VOCAB.stop == 1].sample(10)


,term_str,n,num,stop
term_id,,,,
3978,under,7,0,1
4178,which,98,0,1
3784,that,349,0,1
2328,me,110,0,1
2578,on,164,0,1
4136,we,36,0,1
4302,you,244,0,1
1139,during,2,0,1
2289,m,7,0,1


In [26]:
from nltk.stem.porter import PorterStemmer
stemmer1 = PorterStemmer()
VOCAB['stem_porter'] = VOCAB.term_str.apply(stemmer1.stem)

from nltk.stem.snowball import SnowballStemmer
stemmer2 = SnowballStemmer("english")
VOCAB['stem_snowball'] = VOCAB.term_str.apply(stemmer2.stem)

from nltk.stem.lancaster import LancasterStemmer
stemmer3 = LancasterStemmer()
VOCAB['stem_lancaster'] = VOCAB.term_str.apply(stemmer3.stem)

In [27]:
def get_POS_Count(row):
    term = row['term_str']

    return TOKEN[TOKEN['term_str']== term].value_counts('pos').idxmax() 


In [28]:
VOCAB['pos_max'] = VOCAB.apply(get_POS_Count, axis=1)

In [29]:
VOCAB.sample(10)

,term_str,n,num,stop,stem_porter,stem_snowball,stem_lancaster,pos_max
term_id,,,,,,,,
4190,whomsoever,1,0,0,whomsoev,whomsoev,whomsoev,VB
187,assured,1,0,0,assur,assur,ass,VBD
4006,unlike,2,0,0,unlik,unlik,unlik,IN
3569,stand,3,0,0,stand,stand,stand,VB
2912,public,1,0,0,public,public,publ,JJ
1999,instance,1,0,0,instanc,instanc,inst,NN
3265,sensible,2,0,0,sensibl,sensibl,sens,JJ
3182,sank,2,0,0,sank,sank,sank,NN
3050,remember,5,0,0,rememb,rememb,rememb,VB


## Part 3: Create

In [30]:
count_method = 'n' # 'c' or 'n' # n = n tokens, c = distinct token (term) count
tf_method = 'sum' # sum, max, log, double_norm, raw, binary
tf_norm_k = .5 # only used for double_norm
idf_method = 'standard' # standard, max, smooth
gradient_cmap = 'YlGnBu' # YlGn, GnBu, YlGnBu; For tables; see https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html 

In [31]:
SENTS = OHCO[:4]
PARAS = OHCO[:3]
CHAPS = OHCO[:2]
BOOKS = OHCO[:1]

In [35]:
TOKEN['term_id'] = TOKEN.term_str.map(VOCAB.reset_index().set_index('term_str').term_id)

## Part 4: Extend

## Part 5: Present